# Generalized Forum Scraping

In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import nltk
import gensim

C:\Users\Meret\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Be careful to run the cells in order

Function that finds the tags allowing to retrieve the data we need

In [2]:
def find_tag(x, comp, tags):
    tag = x.name
    try:
        class_name = x['class'][0]
        tags.append(class_name)
        tags.append(tag)
        next_ = x.findNext(tag, {'class': class_name})
    except:
        #print("TAG: ", tag)
        tags.append(tag)
        #print("TAGS: ", tags)
        next_ = x.findNext(tag)
        #print("next: ", next_)
        
    if comp in next_.text:
        print("I've found the right tag, it's ", tags[::-1])
        return tags[::-1]
    else:
        return find_tag(x.parent, comp, tags)

In [3]:
def find_next_tag(x, soup, tag):
    if x != None:
        next_tag = x.parent.name
        try:
            soup.find_all(next_tag, string="Next")[0]['href'][1:]
            tag.append(next_tag)
            print("The tag for next page is ", next_tag)
        except:
            find_next_tag(x.parent, soup, tag)
        return tag[0]
    else:
        print("tag of the next page is not treated in this program")
        return None

In [93]:
def find_next(soup, next_tag):
    try:
        #check if find instead of find_all always work if not do [0]['href'][1:]
        if next_tag is not None:
            return soup.find_all(next_tag, string="Next")[0]['href'][1:]
        return ""
    except:
        return ""

def collect_forum_data(soup_row, tags, soup_row_ok = 0, link=""):
    
    data = {}
    try:
        if not soup_row_ok:
            soup_row = soup_row.find(tags[len(tags) -1])
    
        data['Title'] = soup_row.text.strip()
        data['Link'] = soup_row['href'][1:]
        
    except:
        pass
    return data

def collect_post_data(soup_row, tags, soup_row_ok, msg_or_user, link=""):
    data = {}
    try:        
        if len(tags) == 2 and not soup_row_ok:
            soup_row = soup_row.find(tags[len(tags)-1])
        if len(tags) == 3:                       
            soup_row = soup_row.find(tags[len(tags)-1])
        elif len(tags) == 4:
            soup_row = soup_row.find(tags[2]).find(tags[3])
        
        data['Link'] = link.replace(PREFIX_URL, '')
        data[msg_or_user] = soup_row.text.strip()
        
    except:
        pass
    return data

def collect_recursively(data, soup, tags, next_tag, fcte_name, msg_or_user="", link="", index=""):
    try:
        if index:
            print(index, end='\r', flush=True)
        
        soup_row_ok = 0
        if len(tags) == 1:
            soup_rows = soup.find_all(tags[0])
            soup_row_ok = 1
        if len(tags) == 2:
            soup_rows = soup.find_all(tags[0], {'class': tags[1]})
            if not soup_rows:
                soup_rows = soup.find_all(tags[0]) 
            else:
                soup_row_ok = 1      
        if len(tags) == 3 or len(tags) == 4:
            soup_rows = soup.find_all(tags[0], {'class': tags[1]})
    
        #print(soup_rows)
        data.extend([fcte_name(soup_row, tags, soup_row_ok, msg_or_user, link) for soup_row in soup_rows])
        
        next_url = find_next(soup, next_tag)
        if next_url:
            soup = BeautifulSoup(requests.get(PREFIX_URL + next_url).text, 'html.parser')
            if index:
                return collect_recursively(data, soup, tags, next_tag, fcte_name, msg_or_user, link, index+1)
            else:
                return collect_recursively(data, soup, tags, next_tag, msg_or_user, fcte_name, link)
        else:
            return data
    except:
        pass
    return data
                
def verify_if_treated(soup, tags):
    if len(tags) > 4 or len(tags) < 1:
        print("This case is not treated yet")
        
    if len(tags) == 3 or len(tags) == 4:
        try:
            soup.find(tags[0], {'class': tags[1]})
        except:
            print("This case is not treated yet")
    
def collect_all_links(soup, tags, next_tag, fcte_name):      #soup or url? Combine (put verifications also to collect)
    verify_if_treated(soup, tags)

    data = collect_recursively([], soup, tags, next_tag, fcte_name)
    return pd.DataFrame(data).dropna()

def collect(forum_df, tags, fcte_name, msg_or_user):
    data = []
    total = len(forum_df['Link'])
    index = 0
    for url in forum_df['Link']:
        index += 1
        print('{} out of {}'.format(index, total), end='\r', flush=True)
        #url = forum_df['Link'][0]
        soup = BeautifulSoup(requests.get(PREFIX_URL + url).text, 'html.parser')
        verify_if_treated(soup, tags)
        data.extend(collect_recursively([], soup, tags, next_tag, fcte_name, msg_or_user, url))
    return pd.DataFrame(data)

### 1. Collect all titles and links of the forum

Enter the 3 parameters needed:

In [80]:
#Holiday Truths
PREFIX_URL = 'https://www.holidaytruths.co.uk/'
START_URL = PREFIX_URL + 'forum/america-canada-discussion-forum-f2-0.html'
soup = BeautifulSoup(requests.get(START_URL).text, 'html.parser')
title1 = 'ESTA question on employment'
title2 = 'Vegas Buffets/Restaurants'

#Wrong Planet
#PREFIX_URL = 'http://wrongplanet.net/forums'
#START_URL = PREFIX_URL + '/viewforum.php?f=19'
#soup = BeautifulSoup(requests.get(START_URL).text, 'html.parser')
#title1 = 'RE: Kids w/ Classic Autism, PDD-NOS & Speech Delays'
#title2 = 'Parents on the spectrum'

#Stack Overflow
#PREFIX_URL = 'https://stackoverflow.com/'
#START_URL = PREFIX_URL + 'questions/tagged/forum'
#soup = BeautifulSoup(requests.get(START_URL).text, 'html.parser')
#title1 = 'Should DynamoDB adjacency lists use discrete partition keys to model each type of relationship?'
#title2 = 'How to Bypass [hide] element in Forums?'

#Au Féminin
#PREFIX_URL = 'https://astrologie.aufeminin.com/forum'
#START_URL = PREFIX_URL + '/all'
#soup = BeautifulSoup(requests.get(START_URL).text, 'html.parser')
#title1 = 'coucou....'
#title2 = 'échange serieux en mp'


#Trip Advisor
#PREFIX_URL = 'https://www.tripadvisor.co.uk/'
#START_URL = PREFIX_URL + 'ShowForum-g1-i12334-Holiday_Travel.html'
#soup = BeautifulSoup(requests.get(START_URL).text, 'html.parser')
#title1 = 'See TOP QUESTIONS before posting!'
#title2 = 'Use the SEARCH BOX function before posting!'

Find the tags to be able to scrape the titles and links and the tag needed to find the next page

In [81]:
import re
text = soup.find(text=re.compile(title1))
if text is None:
    print("Unable to scrape this text from this forum")
else:
    right_tags = find_tag(text.parent, title2, [])
    next_tag = find_next_tag(soup.find(text=re.compile("Next")), soup, [])
    print(next_tag)

I've found the right tag, it's  ['span', 'title', 'a']
The tag for next page is  a
a


Scrape all titles and links

In [82]:
threads = collect_all_links(soup, right_tags, next_tag, collect_forum_data)
#print(threads)

### 2. Collect all usernames, messages of every posts in every link

Enter the parameters needed:

In [90]:
START_URL = PREFIX_URL + threads.Link[0]
soup =  BeautifulSoup(requests.get(START_URL).text, 'html.parser')
#print(soup.prettify())

#Holiday Truths
user1 = 'AnnaM'
user2 = 'Glynis HT Admin'
msg1 = 'Hi, I am brand new and hopefully I have put this question in the right area.'
msg2 = 'd have thought that was fine. Plenty of retired people travel.'

#Wrong Planet
#user1 = 'cyberdad'
#user2 = 'Solvejg'

#Trip Advisor
#print(soup.prettify())
#user1 = 'BradJill'
#user2 = 'Eden7'

Find the tags to be able to scrape the usernames

In [96]:
import re
text = soup.find(text=re.compile(user1))
if text is None:
    print("Unable to scrape this text from this forum")
else:
    user_tags = find_tag(text.parent, user2, [])

I've found the right tag, it's  ['div', 'user-name', 'a']


Scrape all usernames

In [97]:
user_posts = collect(threads, user_tags, collect_post_data, 'Username')
#print(user_posts)

                                                    Link         Username
0         forum/esta-question-on-employment-t172445.html            AnnaM
1         forum/esta-question-on-employment-t172445.html  Glynis HT Admin
2         forum/esta-question-on-employment-t172445.html            AnnaM
3         forum/esta-question-on-employment-t172445.html   Lance Chambers
4         forum/esta-question-on-employment-t172445.html            AnnaM
5         forum/esta-question-on-employment-t172445.html            Fiona
6         forum/esta-question-on-employment-t172445.html            Fiona
7         forum/esta-question-on-employment-t172445.html            AnnaM
8         forum/esta-question-on-employment-t172445.html            Fiona
9         forum/esta-question-on-employment-t172445.html            AnnaM
10        forum/esta-question-on-employment-t172445.html            Fiona
11        forum/esta-question-on-employment-t172445.html   James Fletcher
12           forum/vegas-buffets-resta

Find the tags to be able to scrape the messages

In [85]:
import re
text = soup.find(text=re.compile(msg1))
if text is None:
    print("Unable to scrape this text from this forum")
else:
    message_tags = find_tag(text.parent, msg2, [])

I've found the right tag, it's  ['div', 'col-md-10']


Scrape all messages

In [94]:
message_posts = collect(threads, message_tags, collect_post_data, 'Message')

In [107]:
print(user_posts)

                                                    Link         Username
0         forum/esta-question-on-employment-t172445.html            AnnaM
1         forum/esta-question-on-employment-t172445.html  Glynis HT Admin
2         forum/esta-question-on-employment-t172445.html            AnnaM
3         forum/esta-question-on-employment-t172445.html   Lance Chambers
4         forum/esta-question-on-employment-t172445.html            AnnaM
5         forum/esta-question-on-employment-t172445.html            Fiona
6         forum/esta-question-on-employment-t172445.html            Fiona
7         forum/esta-question-on-employment-t172445.html            AnnaM
8         forum/esta-question-on-employment-t172445.html            Fiona
9         forum/esta-question-on-employment-t172445.html            AnnaM
10        forum/esta-question-on-employment-t172445.html            Fiona
11        forum/esta-question-on-employment-t172445.html   James Fletcher
12           forum/vegas-buffets-resta

# Merge the usernames and messages to have all posts

In [98]:
posts = pd.merge(user_posts, message_posts, on='Link', how='inner')

In [111]:
posts

,Link,Username,Message
0,forum/esta-question-on-employment-t172445.html,AnnaM,"Hi, I am brand new and hopefully I have put th..."
1,forum/esta-question-on-employment-t172445.html,AnnaM,Hi Anita & \nIf you are retired Anna then I'...
2,forum/esta-question-on-employment-t172445.html,AnnaM,"Thanks for your input, My worry is if I put No..."
3,forum/esta-question-on-employment-t172445.html,AnnaM,This is not the case - they are more concerned...
4,forum/esta-question-on-employment-t172445.html,AnnaM,They also want to know your parents names? My...
5,forum/esta-question-on-employment-t172445.html,AnnaM,"Anna- if you look at that section, the only co..."
6,forum/esta-question-on-employment-t172445.html,AnnaM,Parents names are pretty normal information ne...
7,forum/esta-question-on-employment-t172445.html,AnnaM,"Hi Fiona, I’ve decided to do a tour called The..."
8,forum/esta-question-on-employment-t172445.html,AnnaM,Is that the Cosmos one? Haven't been to LA but...
9,forum/esta-question-on-employment-t172445.html,AnnaM,"Hi Fiona, yes it’s with Cosmos, I initially wa..."


In [109]:
merged_df = pd.merge(threads, posts, on='Link', how='inner')
merged_df.to_json('C:/Users/Meret/Documents/EPFL/3Annee/Semestre_5/Projet/Forum_Chatbot/Forum Data/holiday_truths_threads_myscraping.json')

In [110]:
merged_df

,Link,Title,Username,Message
0,forum/esta-question-on-employment-t172445.html,ESTA question on employment,AnnaM,"Hi, I am brand new and hopefully I have put th..."
1,forum/esta-question-on-employment-t172445.html,ESTA question on employment,AnnaM,Hi Anita & \nIf you are retired Anna then I'...
2,forum/esta-question-on-employment-t172445.html,ESTA question on employment,AnnaM,"Thanks for your input, My worry is if I put No..."
3,forum/esta-question-on-employment-t172445.html,ESTA question on employment,AnnaM,This is not the case - they are more concerned...
4,forum/esta-question-on-employment-t172445.html,ESTA question on employment,AnnaM,They also want to know your parents names? My...
5,forum/esta-question-on-employment-t172445.html,ESTA question on employment,AnnaM,"Anna- if you look at that section, the only co..."
6,forum/esta-question-on-employment-t172445.html,ESTA question on employment,AnnaM,Parents names are pretty normal information ne...
7,forum/esta-question-on-employment-t172445.html,ESTA question on employment,AnnaM,"Hi Fiona, I’ve decided to do a tour called The..."
8,forum/esta-question-on-employment-t172445.html,ESTA question on employment,AnnaM,Is that the Cosmos one? Haven't been to LA but...
9,forum/esta-question-on-employment-t172445.html,ESTA question on employment,AnnaM,"Hi Fiona, yes it’s with Cosmos, I initially wa..."


In [100]:
original_merged_df = pd.read_json('C:/Users/Meret/Documents/EPFL/3Annee/Semestre_5/Projet/Forum_Chatbot/Forum Data/holiday_truths_threads.json')

In [103]:
original_merged_df = original_merged_df.drop(['Replies', 'Timestamp'], axis=1)

In [104]:
original_merged_df

,Link,Message,Title,Username
0,forum/esta-question-on-employment-t172445.html,"Hi, I am brand new and hopefully I have put th...",ESTA question on employment,AnnaM
1,forum/esta-question-on-employment-t172445.html,Hi Anita & \nIf you are retired Anna then I'...,ESTA question on employment,Glynis HT Admin
10,forum/esta-question-on-employment-t172445.html,You will love it!\n\n \n\n ...,ESTA question on employment,Fiona
100,forum/trip-of-lifetime-t171927.html,I havent done a multi centre in the States apa...,Trip of a lifetime!,Fiona
1000,forum/thanks-for-your-help-t151857.html,Re: Thanks for your help\n\n\n ...,Thanks for your help,luci HT Mod
10000,forum/anyone-ever-used-vrbo-t63592.html,Thanks widget.\n\n \n\n ...,Anyone ever used VRBO,Waterford2005
10001,forum/anyone-ever-used-vrbo-t63592.html,VRBO\n\n\n Just wondering if an...,Anyone ever used VRBO,Waterford2005
10002,forum/anyone-ever-used-vrbo-t63592.html,which company is that?\r\nVrbo?\n\na few have ...,Anyone ever used VRBO,ALISONSVILLA
10003,forum/anyone-ever-used-vrbo-t63592.html,When I had a home in Florida I got about 90% o...,Anyone ever used VRBO,terrynewpack
10004,forum/anyone-ever-used-vrbo-t63592.html,That's really interesting terry-you must have ...,Anyone ever used VRBO,ALISONSVILLA
